<a href="https://colab.research.google.com/github/ducbao811/diveintocode-ml/blob/master/Convolutional2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Problem 1 to 3]

**Preparing the MNIST dataset**

In [1]:
# Importing dataset

from keras.datasets import mnist
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshaping for fitting
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# Coverting value to 0 and 1
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# Splitting data for training and validation
from sklearn.model_selection import train_test_split
X_training, X_val, y_training, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

print("X_tranining shape: " , X_training.shape)
print("X_validation shape: ", X_val.shape)
print("y_tranining shape: ", y_training.shape)
print("y_validation shape: ", y_val.shape)

X_tranining shape:  (48000, 784)
X_validation shape:  (12000, 784)
y_tranining shape:  (48000, 10)
y_validation shape:  (12000, 10)


**Preparing helper class for batch sampling, initializer and optimizer**

In [2]:
class GetMiniBatch:

    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
        
    def __len__(self):
        return self._stop
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    
    def __iter__(self):
        self._counter = 0
        return self
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [3]:
class SimpleInitializer:
    
    def __init__(self, sigma=0.01):
        self.sigma = sigma
        
    def W(self,f_num, Chanel, f_size_h, f_size_w):
        np.random.seed(0)
        self.W = self.sigma * np.random.randn(f_num, Chanel, f_size_h, f_size_w)
        return self.W
    
    def B(self, f_num):
        np.random.seed(0)
        self.B = self.sigma * np.random.randn(f_num,1)
        return self.B

class XavierInitializer:
    
    def __init__(self, sigma = 0.01):
        self.sigma = sigma
        
    def W(self,f_num, Chanel, f_size_h, f_size_w):
        self.sigma = (1 / np.sqrt(f_num))
        np.random.seed(0)
        self.W = self.sigma * np.random.randn(f_num, Chanel, f_size_h, f_size_w)
        return self.W
        
    def B(self,f_num):
        np.random.seed(0)
        self.sigma = (1 / np.sqrt(f_num))
        self.B = self.sigma * np.random.randn(f_num,1)
        return self.B

class He:
    
    def __init__(self, sigma = 0.01):
        self.sigma = sigma
    
    def W(self, f_num, Chanel, f_size_h, f_size_w):
        sigma = np.sqrt(2 / 1)
        np.random.seed(0)
        self.W = sigma * np.random.randn(f_num, Chanel, f_size_h, f_size_w)
        return self.W
        
    def B(self,f_num):
        sigma = np.sqrt(2 / 1)
        np.random.seed(0)
        self.B = sigma * np.random.randn(f_num,1)
        return self.B

class SGD:

    def __init__(self, lr=0.001):
        self.lr = lr
        
    def update(self,layer):

        layer.W -= (self.lr * layer.dW)
        layer.B -= (self.lr * layer.dB)
        return layer.W, layer.B

**Calculation of width and height of output**

In [15]:
# Height calculation
def output_heigh_calculation(X_h,pad_n,F_h,stride):
    output_h = (X_h + 2*pad_n - F_h)//stride + 1
    return output_h

# Width calculation
def output_width_calculation(X_w,pad_n,F_w,stride):
    output_w = (X_w + 2*pad_n - F_w)//stride + 1
    return output_w

**2D 2-D convolutional layer**

In [4]:
class SimpleConv2d:
    
    def __init__(self, f_num=1, f_size_h=2, f_size_w=2, stride=1, pad = 1, initializer=XavierInitializer(), optimizer=SGD(lr=0.05)):

        self.f_num = f_num
        self.f_size_h = f_size_h
        self.f_size_w = f_size_w
        self.pad = pad
        self.stride = stride
        self.kernel  = None
        self.initializer = initializer
        self.optimizer = optimizer
        self.W = None
        self.B = None 
        
    def im2col(self, input_data):
        N, C, H, W = input_data.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h= self.f_size_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.f_size_w, stride=self.stride)
        img = np.pad(input_data, [(0,0), (0,0), (self.pad, self.pad), (self.pad, self.pad)], 'constant')
        col = np.zeros((N, C, self.f_size_h, self.f_size_w, out_h, out_w))

        for y in range(self.f_size_h):
            y_max = y + self.stride*out_h
            for x in range(self.f_size_w):
                x_max = x + self.stride*out_w
                col[:, :, y, x, :, :] = img[:, :, y:y_max:self.stride, x:x_max:self.stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col
    
    def col2im(self, col, X):
        N, C, H, W = X.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h=self.f_size_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.f_size_w, stride=self.stride)
        col = col.reshape(N, out_h, out_w, C, self.f_size_h, self.f_size_w).transpose(0, 3, 4, 5, 1, 2)
        img = np.zeros((N, C, H + 2*self.pad + self.stride - 1, W + 2*self.pad + self.stride - 1))
        
        for y in range(self.f_size_h):
            y_max = y + self.stride*out_h
            for x in range(self.f_size_w):
                x_max = x + self.stride*out_w
                img[:, :, y:y_max:self.stride, x:x_max:self.stride] += col[:, :, y, x, :, :]
        return img[:, :, self.pad:H + self.pad, self.pad:W + self.pad]
        
    def forward(self,X):
        if self.W is None:
            self.W = self.initializer.W(self.f_num, X.shape[1], self.f_size_h , self.f_size_w)
        if self.B is None:
            self.B = self.initializer.B(self.f_num)
        FN, C, FH, FW = self.W.shape
        N, C, H, W = X.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h=self.f_size_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.f_size_w, stride=self.stride)
            
        self.col = self.im2col(X)
        self.col_W = self.W.reshape(FN, -1).T
        out = np.dot(self.col, self.col_W) + self.B.T
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        self.X = X
        return out
    
    def backward(self,dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)
        self.dB = np.sum(dout, axis=0).reshape(self.f_num,1)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        self.dx = self.col2im(dcol,self.X)
        
        return self.dx
        self = self.optimizer.update(self)

In [ ]:
test = SimpleConv2d(f_num=4, f_size_h=3,  f_size_w=3, stride=1, pad=1, initializer=SimpleInitializer())
get_mini_batch = GetMiniBatch(X_train, y_train_one_hot, batch_size=20)
for mini_X_train, mini_y_train in get_mini_batch:
    mini_X_train, mini_y_train
mini_X_train = mini_X_train.reshape(20, 1, 28, 28)
testing = test.forward(mini_X_train)

In [ ]:
test.backward(testing)

array([[[[ 1.68067009e-03,  1.56469796e-03,  1.56469796e-03, ...,
           1.56469796e-03,  1.56469796e-03,  7.23931309e-04],
         [ 1.45267581e-03,  1.34980047e-03,  1.34980047e-03, ...,
           1.34980047e-03,  1.34980047e-03,  5.49806859e-04],
         [ 1.45267581e-03,  1.34980047e-03,  1.34980047e-03, ...,
           1.34980047e-03,  1.34980047e-03,  5.49806859e-04],
         ...,
         [ 1.45267581e-03,  1.34980047e-03,  1.34980047e-03, ...,
           1.34980047e-03,  1.34980047e-03,  5.49806859e-04],
         [ 1.45267581e-03,  1.34980047e-03,  1.34980047e-03, ...,
           1.34980047e-03,  1.34980047e-03,  5.49806859e-04],
         [ 8.00122436e-04,  3.87001988e-04,  3.87001988e-04, ...,
           3.87001988e-04,  3.87001988e-04, -9.66779355e-05]]],


       [[[ 1.68067009e-03,  1.56469796e-03,  1.56469796e-03, ...,
           1.56469796e-03,  1.56469796e-03,  7.23931309e-04],
         [ 1.45267581e-03,  1.34980047e-03,  1.34980047e-03, ...,
           1.3498004

In [ ]:
test.backward(testing).shape

(20, 1, 28, 28)

# [Problem 4]

In [5]:
class MaxPool2D:
    
    def __init__(self, P_h, P_w, stride=1, pad=0):
        self.P_h = P_h
        self.P_w = P_w
        self.stride = stride
        self.pad = pad
        self.x = None
        self.arg_max = None
        
    def im2col(self, input_data):
        N, C, H, W = input_data.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h= self.P_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.P_w, stride=self.stride)
        img = np.pad(input_data, [(0,0), (0,0), (self.pad, self.pad), (self.pad, self.pad)], 'constant')
        col = np.zeros((N, C, self.P_h, self.P_w, out_h, out_w))

        for y in range(self.P_h):
            y_max = y + self.stride*out_h
            for x in range(self.P_w):
                x_max = x + self.stride*out_w
                col[:, :, y, x, :, :] = img[:, :, y:y_max:self.stride, x:x_max:self.stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col
    
    def col2im(self, col, X):
        N, C, H, W = X.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h=self.P_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.P_w, stride=self.stride)
        col = col.reshape(N, out_h, out_w, C, self.P_h, self.P_w).transpose(0, 3, 4, 5, 1, 2)
        img = np.zeros((N, C, H + 2*self.pad + self.stride - 1, W + 2*self.pad + self.stride - 1))
        
        for y in range(self.P_h):
            y_max = y + self.stride*out_h
            for x in range(self.P_w):
                x_max = x + self.stride*out_w
                img[:, :, y:y_max:self.stride, x:x_max:self.stride] += col[:, :, y, x, :, :]
        return img[:, :, self.pad:H + self.pad, self.pad:W + self.pad]
       
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h=self.P_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.P_w, stride=self.stride)
        col = self.im2col(x)     
        col = col.reshape(-1, self.P_h*self.P_w)
        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        self.x = x
        self.arg_max = arg_max
        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)    
        p_size = self.P_h * self.P_w
        dmax = np.zeros((dout.size, p_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (p_size,))     
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = self.col2im(dcol, self.x)
        return dx
        self = self.optimizer.update(self)

In [ ]:
# Testing max pooling layer
pooling_layer = MaxPool2D(P_h= 2, P_w = 2, stride= 2, pad=0)
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

In [ ]:
out_pooling = pooling_layer.forward(x)
backward_pooling = pooling_layer.backward(out_pooling)

In [ ]:
print(out_pooling)

[[[[ 6.  8.]
   [14. 16.]]]]


In [ ]:
print(backward_pooling)

[[[[ 0.  0.  0.  0.]
   [ 0.  6.  0.  8.]
   [ 0.  0.  0.  0.]
   [ 0. 14.  0. 16.]]]]


# [Problem 5]

In [ ]:
class AvgPool2D:
    
    def __init__(self, P_h, P_w, stride=1, pad=0):
        self.P_h = P_h
        self.P_w = P_w
        self.stride = stride
        self.pad = pad
        self.x = None
        
    def im2col(self, input_data):
        N, C, H, W = input_data.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h= self.P_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.P_w, stride=self.stride)
        img = np.pad(input_data, [(0,0), (0,0), (self.pad, self.pad), (self.pad, self.pad)], 'constant')
        col = np.zeros((N, C, self.P_h, self.P_w, out_h, out_w))

        for y in range(self.P_h):
            y_max = y + self.stride*out_h
            for x in range(self.P_w):
                x_max = x + self.stride*out_w
                col[:, :, y, x, :, :] = img[:, :, y:y_max:self.stride, x:x_max:self.stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col
    
    def col2im(self, col, X):
        N, C, H, W = X.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h=self.P_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.P_w, stride=self.stride)
        col = col.reshape(N, out_h, out_w, C, self.P_h, self.P_w).transpose(0, 3, 4, 5, 1, 2)
        img = np.zeros((N, C, H + 2*self.pad + self.stride - 1, W + 2*self.pad + self.stride - 1))
        
        for y in range(self.P_h):
            y_max = y + self.stride*out_h
            for x in range(self.P_w):
                x_max = x + self.stride*out_w
                img[:, :, y:y_max:self.stride, x:x_max:self.stride] += col[:, :, y, x, :, :]
        return img[:, :, self.pad:H + self.pad, self.pad:W + self.pad]
       
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = output_heigh_calculation(X_h=H,pad_n=self.pad, F_h=self.P_h, stride=self.stride)
        out_w = output_width_calculation(X_w=W,pad_n=self.pad, F_w=self.P_w, stride=self.stride)
        col = self.im2col(x)     
        col = col.reshape(-1, self.P_h*self.P_w)
        out = np.average(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        self.x = x
        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)    
        p_size = self.P_h * self.P_w
        dmax = np.full((dout.size, p_size), 1 / p_size)
        dmax = dmax.reshape(dout.shape + (p_size,))     
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = self.col2im(dcol, self.x)
        return dx
        self = self.optimizer.update(self)

In [ ]:
# Testing average pooling layer
pooling_layer = AvgPool2D(P_h= 2, P_w = 2, stride= 2, pad=0)
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

In [ ]:
out_pooling = pooling_layer.forward(x)
backward_pooling = pooling_layer.backward(out_pooling)

In [ ]:
print(out_pooling)

[[[[ 3.5  5.5]
   [11.5 13.5]]]]


In [ ]:
print(backward_pooling)

[[[[0.25 0.25 0.25 0.25]
   [0.25 0.25 0.25 0.25]
   [0.25 0.25 0.25 0.25]
   [0.25 0.25 0.25 0.25]]]]


# [Problem 6]

In [13]:
class Flatten:
    
    def forward(self, X):
        N, C, H, W = X.shape
        out = X.reshape(N, -1)        
        self.X = X        
        return out
    
    def backward(self, dout):
        dx = dout.reshape(self.X.shape)      
        return dx

In [ ]:
smoothing = Flatten()
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

In [ ]:
smoothing_forward = smoothing.forward(x)
smoothing_backward = smoothing.backward(smoothing_forward)

In [ ]:
print("Initial shape: ", x.shape)
print("Forward shape: ", smoothing_forward.shape)
print("Backward shape: ", smoothing_backward.shape)

Initial shape:  (1, 1, 4, 4)
Forward shape:  (1, 16)
Backward shape:  (1, 1, 4, 4)


# [Problem 7]

**Define some class needed**

In [6]:
# Fully-connected layer
class FC:

    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = initializer.B(self.n_nodes2)
        self.optimizer = optimizer
        self.HW = 0
        self.HB = 0
        
    def forward(self, X): 
        self.Z = X
        self.A = X @ self.W + self.B
        return self.A
    
    def backward(self, dA):
        self.dB = np.sum(dA, axis=0)
        self.dW = self.Z.T @ dA
        self.dZ = dA @ self.W.T
        self = self.optimizer.update(self)
        return self.dZ

In [7]:
# Optimizer and Activater
class Sigmoid:
    
    def forward(self, A):
        self.A = A
        return self.sigmoid(A)
    
    def backward(self, dZ):
        _sig = self.sigmoid(self.A)
        return dZ * (1 - _sig)*_sig
    
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

class Tanh:
    
    def forward(self, A):
        self.A = A
        return np.tanh(A)
    
    def backward(self, dZ):
        return dZ * (1 - (np.tanh(self.A))**2)

class Softmax:
    
    def forward(self, A): 
        Z = np.exp(A) / np.sum(np.exp(A), axis=1).reshape(-1, 1)
        return Z
        
    def backward(self, Z, y):
        dA = Z - y
        loss = - np.sum(y * np.log(Z)) / len(y)
        return dA, loss

class AdaGrad:

    def __init__(self, lr):
        self.lr = lr 
    
    def update(self, layer):
        layer.HW += layer.dW * layer.dW
        layer.HB += layer.dB * layer.dB
        delta = 1e-7 
        layer.W -= self.lr * layer.dW / (np.sqrt(layer.HW) + delta) / len(layer.Z)
        layer.B -= self.lr * layer.dB / (np.sqrt(layer.HB) + delta) / len(layer.Z)
        return layer

class HeInitializer:

    def __init__(self, sigma):
        _ = sigma
        
    def W(self, n_nodes1, n_nodes2):
        self.sigma = np.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(1, n_nodes2)
        return B

class ReLU:

    def forward(self, A):
        self.A = A
        Z = np.maximum(0, A)
        return Z
    
    def backward(self, dZ):
        dA = dZ * np.where(self.A > 0, 1, 0)
        return dA

In [8]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = output_heigh_calculation(X_h=H,pad_n=pad, F_h=filter_h, stride=stride)
    out_w = output_width_calculation(X_w=W,pad_n=pad, F_w=filter_w, stride=stride)
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

class MaxPool2D2:

    def __init__(self, P_h, P_w, pad=0):
        self.P_h = P_h
        self.P_w = P_w
        self.stride = P_h
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.P_h) / self.stride)
        out_w = int(1 + (W - self.P_w) / self.stride)
        col = im2col(x, self.P_h, self.P_w, self.stride, self.pad)
        col = col.reshape(-1, self.P_h*self.P_w)
        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        self.x = x
        self.arg_max = arg_max
        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)    
        pool_size = self.P_h * self.P_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,))       
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.P_h, self.P_w, self.stride, self.pad)    
        return dx

In [9]:
class Scratch2dCNNClassifier:
    
    """
    2-layer class for 2D convolutional networks 
    """

    def __init__(self, FN=20, FH=7, FW=7, stride=1, pad=0, epoch=1, optimizer=AdaGrad, initializer=HeInitializer, activater=ReLU, verbose=False,):
        self.verbose = verbose
        self.batch_size = 60 
        self.n_output = 10 
        self.sigma = 0.02 
        self.lr = 0.05 
        self.epoch = epoch 
        self.optimizer = optimizer 
        self.initializer = initializer 
        self.activater = activater 
        self.FN = FN 
        self.C = 1 
        self.FH = FH 
        self.FW = FW 
        self.pool_h = 2 
        self.pool_w = 2 
        self.pad = pad 
        self.stride = stride 
    
    def fit(self, X, y, X_val=None, y_val=None):
        self.loss_train = [] 
        self.loss_val = []   
        out_h, out_w = self.out_size(28, 28, self.pad, self.FH, self.FW, self.stride)
        out_h, out_w = self.out_size(out_h, out_w, 0, self.pool_h, self.pool_w, self.pool_h)
        fc_nodes = self.FN * out_h * out_w 
        optimizer = self.optimizer(self.lr)    
        w = self.sigma * np.random.randn(self.FN, self.C, self.FH, self.FW)
        b = self.sigma * np.random.randn(self.FN,)
        self.cv= SimpleConv2d(self.FN, self.FH, self.FW, self.stride, self.pad, initializer=SimpleInitializer())
        self.activation_cv = self.activater()
        self.pl = MaxPool2D2(self.pool_h, self.pool_w)
        self.fl = Flatten()
        self.FC = FC(fc_nodes, self.n_output, self.initializer(self.sigma), optimizer)
        self.activation_fc = Softmax()

        for i in range(self.epoch):
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size, seed=i)
            for mini_X, mini_y in get_mini_batch:
                
                A1 = self.cv.forward(mini_X)
                Z1 = self.activation_cv.forward(A1)
                P1 = self.pl.forward(Z1)
                F1 = self.fl.forward(P1)
                A2 = self.FC.forward(F1)
                Z2 = self.activation_fc.forward(A2)

                dA2, loss = self.activation_fc.backward(Z2, mini_y) 
                dZ2 = self.FC.backward(dA2)
                dF1 = self.fl.backward(dZ2)
                dP1 = self.pl.backward(dF1)
                dA1 = self.activation_cv.backward(dP1)
                dZ1 = self.cv.backward(dA1)

            if self.verbose:
                A1 = self.cv.forward(X)
                Z1 = self.activation_cv.forward(A1)
                P1 = self.pl.forward(Z1)
                F1 = self.fl.forward(P1)
                A2 = self.FC.forward(F1)
                Z2 = self.activation_fc.forward(A2)  
                self.loss_train.append(self.activation_fc.backward(Z2, y)[1])
                print("Epoch {}, train_loss : {}".format(i , self.loss_train[-1]))
                
                if X_val is not None:
                    A1 = self.cv.forward(X_val)
                    Z1 = self.activation_cv.forward(A1)
                    P1 = self.pl.forward(Z1)
                    F1 = self.fl.forward(P1)
                    A2 = self.FC.forward(F1)
                    Z2 = self.activation_fc.forward(A2)         
                    self.loss_val.append(self.activation_fc.backward(Z2, y_val)[1])
                    print("Epoch {}, validation_loss : {}".format(i , self.loss_val[-1]))
                    
    def out_size(self, H, W, P, FH, FW, S):
        out_h = (H + 2 * P - FH) // S + 1
        out_w = (W + 2 * P - FW) // S + 1
        return out_h, out_w
    
    def predict(self, X):
        A1 = self.cv.forward(X)
        Z1 = self.activation_cv.forward(A1)
        P1 = self.pl.forward(Z1)
        F1 = self.fl.forward(P1)
        A2 = self.FC.forward(F1)
        Z2 = self.activation_fc.forward(A2)
        return np.argmax(Z2, axis=1)

**Training and estimation**

In [10]:
from keras.datasets import mnist
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
(X_train2, y_train2), (X_test2, y_test2) = mnist.load_data()

X_train2 = X_train2.astype(np.float)
X_test2 = X_test2.astype(np.float)
X_train2 /= 255 
X_test2 /= 255

X_train2 = X_train2[:, np.newaxis, :, :] 
X_test2 = X_test2[:, np.newaxis, :, :]

X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size=0.2)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot2 = enc.fit_transform(y_train2[:, np.newaxis])
y_test_one_hot2 = enc.transform(y_val2[:, np.newaxis])

In [16]:
model = Scratch2dCNNClassifier(FN=5, FH=7, FW=7, stride=1, pad=3, epoch = 10, verbose=True)
model.fit(X_train2[:5000], y_train_one_hot2[:5000] )

Epoch 0, train_loss : 2.2482775361933083
Epoch 1, train_loss : 2.224434594743635
Epoch 2, train_loss : 2.2062941710676593
Epoch 3, train_loss : 2.1911365497947934
Epoch 4, train_loss : 2.1779289259674113
Epoch 5, train_loss : 2.166089286540588
Epoch 6, train_loss : 2.155289140733587
Epoch 7, train_loss : 2.1452825891880263
Epoch 8, train_loss : 2.1359295188942573
Epoch 9, train_loss : 2.127129072168834


In [20]:
from sklearn.metrics import accuracy_score

training_pred_default = model.predict(X=X_train2[:5000])
test_pred_default = model.predict(X=X_test2[:5000])

print("Accuracy score for training set with default model: {}".format(accuracy_score(y_true=y_train2[:5000], y_pred=training_pred_default)))
print("Accuracy score for testing set with default model: {}".format(accuracy_score(y_true=y_test2[:5000],y_pred=test_pred_default)))

Accuracy score for training set with default model: 0.6518
Accuracy score for testing set with default model: 0.6184


# [Problem 10]

In [21]:
def output_size(H, W, P, FH, FW, S):
    out_h = (H + 2 * P - FH) // S + 1
    out_w = (W + 2 * P - FW) // S + 1
    return out_h, out_w

def number_of_parameters(FH, FW, C, FN):
    return FH*FW*C*FN + FN

In [22]:
"""
Input size : 144 x 144, 3 channels
Filter size: 3 x 3, 6 channels
Stride : 1
Padding: none
"""
print(output_size(144, 144, 0, 3, 3, 1))
print(number_of_parameters(3, 3, 3, 6))

(142, 142)
168


In [23]:
"""
Input size : 60x60, 24 channels
Filter size: 3 x 3, 48 channels
Stride : 1
Padding: none
"""
print(output_size(60, 60, 0, 3, 3, 1))
print(number_of_parameters(3, 3, 24, 48))

(58, 58)
10416


In [24]:
"""
Input size : 20x20, 10 channels
Filter size: 3 x 3, 20 channels
Stride: 2
Padding: none
"""
print(output_size(20, 20, 0, 3, 3, 2))
print(number_of_parameters(3, 3, 10, 20))

(9, 9)
1820
